In [ ]:
# %pip install pymssql

In [1]:
import pymssql
import pandas as pd
import redis
import time
import datetime
from ipywidgets import IntProgress

In [2]:
REDIS_IP = '10.2.5.212'

In [3]:
def ms_sql_con():
    sql_name = 'voice_ai'
    sql_server = '10.2.4.124'
    sql_login = 'ICECORP\\1c_sql'

    with open('sql.pass','r') as file:
        sql_pass = file.read().replace('\n', '')
        file.close()

    return pymssql.connect(
            server = sql_server,
            user = sql_login,
            password = sql_pass,
            database = sql_name,
            autocommit=True
        )

In [4]:
def read_sql(query):
    return pd.read_sql(query, con=ms_sql_con(), parse_dates=None)

In [109]:
def concatenate_linkedid_side(side, record_date, linkedid):
    query = "SELECT text from transcribations where "
    query += " side="+str(side)+" and "
    query += " record_date = '"+str(record_date)+"' and "
    query += " linkedid = '"+str(linkedid)+"';"
    text_df = read_sql(query)
    phrases_count = len(text_df)
    text_full = ', '.join([row.text for _id, row in text_df.iterrows()])
    return text_full, phrases_count

In [86]:
def summarize(text, phrases_count):
    if phrases_count<2 or len(text)<255:
        return text
    subscriber = redis.StrictRedis(host=REDIS_IP)
    publisher = redis.StrictRedis(host=REDIS_IP) 
    pub = publisher.pubsub()
    sub = subscriber.pubsub()
    sub.subscribe('summarus_client')
    # send
    publisher.publish("summarus_server", text)
    # receive
    while True:
        message = sub.get_message()
        if message and message['type']!='subscribe':
            return message['data'].decode("utf-8")
        time.sleep(1)

In [77]:
def sum_to_sql(linkedid, recor_date, side, text, phrases_count, text_length):
    current_date = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    query = "insert into summarization(linkedid, record_date, sum_date, side, text, phrases_count, text_length) "
    query += " values("
    query += "'"+str(linkedid)+"',"
    query += "'"+str(recor_date)+"',"
    query += "'"+str(current_date)+"',"
    query += str(side)+","
    query += "'"+str(text)+"',"
    query += "'"+str(phrases_count)+"',"
    query += "'"+str(text_length)+"'"
    query += ");"
    #df = read_sql(query)
    
    conn = ms_sql_con()  
    cursor = conn.cursor()
    cursor.execute(query)
    # conn.commit()

In [ ]:
# queue
query = "SELECT column_name FROM information_schema.columns WHERE table_name='queue';"
queue = read_sql(query)
queue

In [ ]:
# transcribations
query = "SELECT column_name FROM information_schema.columns WHERE table_name='transcribations';"
queue = read_sql(query)
queue

### queue linkedid list

In [23]:
# queue
query = "SELECT distinct record_date, linkedid from queue;"
queue = read_sql(query)
len(queue)

0

### transcribations linkedid list

In [ ]:
# queue
query = "SELECT distinct top 10 record_date, linkedid from transcribations;"
df = read_sql(query)
df

In [ ]:
query = "SELECT distinct top 100 trans.record_date, trans.linkedid"
query += " from transcribations as trans"
query += " left join queue as queue"
query += " on trans.record_date=queue.record_date"
query += " and trans.linkedid=queue.linkedid"
query += " where "
query += " trans.record_date > '2021-06-04 00:00:00' and"
query += " isnull(queue.linkedid,'x')='x'"
query += " order by record_date desc;"
%time df = read_sql(query)
len(df)

# start

In [9]:
BATCH_SIZE = 1000
MAX_TEXT_SIZE = 1023

In [29]:
# obtain datetime limits
#query = "select min(record_date) from queue where not isnull(record_date,'2021-06-07 13:34:51')='2021-06-07 13:34:51';"
query = "select linkedid from queue;"
df = read_sql(query)
if len(df):
    query = "select min(record_date) from queue where not isnull(record_date,'')='';"
    df = read_sql(query)
    queue_first_record = str(df.iloc()[0][0])
else:
    queue_first_record = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')

query = "select max(record_date) from summarization;"
df = read_sql(query)
summarization_first_record = str(df.iloc()[0][0])

0

In [30]:
queue_first_record, summarization_first_record

('2021-06-09 08:43:01', '2021-06-07 13:34:51')

In [11]:
query

"SELECT distinct top 1000 record_date, linkedid from transcribations where  record_date < 'None' and not linkedid in (select distinct linkedid from summarization) order by record_date desc;"

In [10]:
query = "SELECT distinct top "+str(BATCH_SIZE)+" record_date, linkedid"
query += " from transcribations"
query += " where "
query += " record_date < '"+queue_first_record+"' and"
query += " not linkedid in (select distinct linkedid from summarization)"
query += " order by record_date desc;"
df = read_sql(query)
len(df)

DatabaseError: Execution failed on sql 'SELECT distinct top 1000 record_date, linkedid from transcribations where  record_date < 'None' and not linkedid in (select distinct linkedid from summarization) order by record_date desc;': (241, b'Conversion failed when converting date and/or time from character string.DB-Lib error message 20018, severity 16:\nGeneral SQL Server error: Check messages from the SQL Server\n')

In [125]:
#text_full

In [126]:
for _id, row in df.iterrows():
    
    for side in range(2):
        text_full, phrases_count = concatenate_linkedid_side(side, row.record_date, row.linkedid)
        text_short = summarize(text_full, phrases_count)
        sum_to_sql(row.linkedid, row.record_date, side, text_short, phrases_count, len(text_full))

In [128]:
len('Российский мастер по ремонту стиральных машин, с кем можно переговорить, пожаловался на то, что белье не попадает в барабан, барабан не крутится, не вращается, он перетащил машинку с одного места на второе, а она почему-то перестала стирать, да, да нет москва, а я давно как ты взял эту самую визитку у этого почтовом ящике и вот по ней и звоню айсберг вот визитка, да конечно номер триста шестнадцать серия, спасибо спасибо, да пожалуй, да.')

441

In [11]:
query = "SELECT * from summarization"
query += " where  record_date = '2021.05.30 11:46:10' and linkedid = '1622364369.119129' and side = '0';"
df = read_sql(query)
df

,linkedid,record_date,sum_date,side,load_msk,load_spb,load_reg,phrases_count,text_length,text,source_id
0,1622364369.119129,2021-05-30 11:46:10,2021-06-16 09:17:36,False,None,None,None,39,1156,"В частном ремонте стиральной машины, в котором...",1


In [53]:
def summarization():
    message = ''
    query = "SELECT count(linkedid) from summarization"
    query += " where isnull(load_msk,0) = 0 or isnull(load_spb,0) = 0 or isnull(load_reg,0) = 0;"
    df = read_sql(query)
    message += '\nСуммаризации, ожидающие загрузку: ' + str(df.iloc()[0][0])
    
    message += '\n== За вчера =='
    
    currentdate = datetime.datetime.today()
    start_of_day = currentdate.combine(currentdate.date(), currentdate.min.time())
    yesterday = start_of_day + datetime.timedelta(days=-1)
    df = str(yesterday)
    dt = str(start_of_day)
    f_0 = '%Y-%m-%d %H:%M:%S'
    f_1 = '%Y-%m-%dT%H:%M:%S'
    date_from = datetime.datetime.strptime(df, f_0).strftime(f_1)
    date_toto = datetime.datetime.strptime(dt, f_0).strftime(f_1)
    
    query = "SELECT count(linkedid) from summarization"
    query += " where sum_date>'" + date_from + "'"
    query += " and sum_date<'" + date_toto + "';"
    df = read_sql(query)
    message += '\nНовых суммаризаций: ' + str(df.iloc()[0][0])
    
    for city in ['load_msk', 'load_spb', 'load_reg']:
        query = "SELECT count(linkedid) from summarization"
        query += " where sum_date>'" + date_from + "'"
        query += " and sum_date<'" + date_toto + "'"
        query += " and isnull("+city+",0) = 1;"
        df = read_sql(query)
        message += '\n'+city+': ' + str(df.iloc()[0][0])      
    
    return message
    
summarization()

'\nСуммаризации, ожидающие загрузку: 92197\n== За вчера ==\nНовых суммаризаций: 6493\nload_msk: 3635\nload_spb: 3563\nload_reg: 3563'

In [39]:
query = "SELECT count(linkedid) from summarization where ;"
df = read_sql(query)
df.iloc()[0][0]

132696

In [31]:
query = "SELECT max(sum_date) from summarization;"
df = read_sql(query)
df

,
0,2021-06-17 13:48:00


In [48]:
query = "SELECT * from summarization order by sum_date desc;"
df = read_sql(query)
min(df.record_date), max(df.record_date)

(Timestamp('2021-05-30 11:44:44'), Timestamp('2021-06-17 11:09:43'))

In [49]:
df

,linkedid,record_date,sum_date,side,load_msk,load_spb,load_reg,phrases_count,text_length,text,source_id
0,1623911864.208392,2021-06-17 09:37:45,2021-06-17 15:34:56,False,None,None,None,8,420,"Гражданин Москвы, проживающий в Москве, просит...",1
1,1623911864.208391,2021-06-17 09:37:45,2021-06-17 15:34:50,True,None,None,None,18,811,"В одной из московских клининговых компаний, ко...",1
2,1623911864.208391,2021-06-17 09:37:45,2021-06-17 15:34:45,False,None,None,None,14,448,"Российские клининговые компании, уборщицы поме...",1
3,1623911865.627322,2021-06-17 09:37:47,2021-06-17 15:34:40,True,None,None,None,17,832,"На днях у нас произошло отключение света, и я ...",2
4,1623911865.627322,2021-06-17 09:37:47,2021-06-17 15:34:34,False,True,None,None,18,952,Российские мастера по укладке плиты отремонтир...,2
5,1623911869.208394,2021-06-17 09:37:50,2021-06-17 15:34:26,True,True,None,None,36,1322,"Позвонив в единую службу сервиса, я поняла, чт...",1
6,1623911869.208394,2021-06-17 09:37:50,2021-06-17 15:34:20,False,True,None,None,31,1154,Накануне одиннадцатого боя окон у меня произош...,1
7,1623911875.627330,2021-06-17 09:37:57,2021-06-17 15:34:14,True,True,True,None,6,215,"и, алло, да да да александр приезжайте я вас ж...",2
8,1623911875.627330,2021-06-17 09:37:57,2021-06-17 15:34:10,False,True,True,None,5,78,"оно да да да, так кольт, два цитата из первая ...",2
9,1623911892.627346,2021-06-17 09:38:15,2021-06-17 15:34:06,True,True,True,True,2,54,"и, номер недоступен оставьте сообщение на авто...",2


In [28]:
query = "SELECT * from summarization order by sum_date desc;"
df = read_sql(query)
#for _id, row in df.iterrows():
    #df.text.iloc()[3]
#    print(row.sum_date, row.record_date)
min(df.record_date), max(df.record_date)

(Timestamp('2021-05-30 11:44:44'), Timestamp('2021-06-17 09:20:08'))

In [22]:
query = "SELECT max(record_date) from transcribations;"
df = read_sql(query)
df

,
0,2021-06-17 07:19:05


In [5]:
min(df.record_date), max(df.record_date)

(Timestamp('2021-06-04 12:04:01'), Timestamp('2021-06-07 13:34:51'))

In [16]:
min(df.record_date), max(df.record_date)

(Timestamp('2021-06-04 12:04:01'), Timestamp('2021-06-07 12:05:32'))

In [46]:
row

NameError: name 'row' is not defined

query = "SELECT column_name FROM information_schema.columns WHERE table_name='summarization';"
queue = read_sql(query)
queue

In [50]:
query = "SELECT linkedid, side, text from summarization where "
query += " linkedid = '1622976650.277074';"
df = read_sql(query)
df

,linkedid,side,text
0,1622976650.277074,False,"здравствуйте скажите пожалуйста сколько будет,..."
1,1622976650.277074,True,"и, иди на фото сервиса оператор, да, кран како..."


In [132]:
# select
query = "SELECT side, text from transcribations where "
query += " record_date = '2021-06-04 12:33:20' and linkedid = '1622799198.236534';"
df = read_sql(query)
', '.join([row.text for _id, row in df.iterrows()])

'здравствуйте скажите пожалуйста мастера можно вызвать по ремонту холодильников сна, москве, стянул, он бы уже давно да, двух, он там где морозильник не сама м-ра телка вот обычно помещаю стал работать как морозилку, да да да, о нет магнитики разброс, вверху справа но мне наверно старая ещё три семнадцать, восемь девятьсот девять, девятьсот восемьдесят один пятьдесят один, семьдесят, галина, улица онежская, дом тридцать пять, корпус два квартира двести пятьдесят семь, третий подъезд пятый этаж без лифта, водный стадион, да, сто пятьдесят девять сорок пять восемьдесят четыре, хорошо спасибо, и, кузбасс сервиса оператору, могу принять заявку в течение часа мастер перезвонит у него все уточнить и подберёте временно визит живете в москве и области, холодильник как называется, гарантия закончилась от производителя холодильник двухкамерные трёхкамерные что не работает, и, промерзание шкафа да да, вы нас стефан узнали где магнит визитка сервисная книжка, вверху справа три цифры какие стоят на

In [133]:
len('здравствуйте скажите пожалуйста мастера можно вызвать по ремонту холодильников сна, москве, стянул, он бы уже давно да, двух, он там где морозильник не сама м-ра телка вот обычно помещаю стал работать как морозилку, да да да, о нет магнитики разброс, вверху справа но мне наверно старая ещё три семнадцать, восемь девятьсот девять, девятьсот восемьдесят один пятьдесят один, семьдесят, галина, улица онежская, дом тридцать пять, корпус два квартира двести пятьдесят семь, третий подъезд пятый этаж без лифта, водный стадион, да, сто пятьдесят девять сорок пять восемьдесят четыре, хорошо спасибо, и, кузбасс сервиса оператору, могу принять заявку в течение часа мастер перезвонит у него все уточнить и подберёте временно визит живете в москве и области, холодильник как называется, гарантия закончилась от производителя холодильник двухкамерные трёхкамерные что не работает, и, промерзание шкафа да да, вы нас стефан узнали где магнит визитка сервисная книжка, вверху справа три цифры какие стоят на магнитных карт сможет отказать, на, спасибо мы указываем адрес телефон в течение час поступит звонок телефон вас скажите, зовут вас как, гален улиц какая у вас, номер дома, корпус есть, квартира, подъезд и этаж, метро какое-то ближайшее, давайте проверим а несколько тридцать пять корпус два квартира два пять семь подъезд третий этаж пятый телефон девятьсот девять девять восемь один пять один семьдесят северном, техника авто номер вашей заявки будет сто пятьдесят девять сорок пять восемьдесят четыре, сорок пять восемьдесят четыре ждите звонок пожалуйста, пожалуйста свидания')

1579

In [135]:
a = 'hello world'
a[:3]

'hel'

In [33]:
#df.text.iloc()[3]

'Germany. Москва, улица матроса железняка дом тридцать шесть квартира шестьдесят семь. подъезд четыре этаж два код домофона четыре пятнадцать. двенадцати до четырнадцати. досками.'

# DELETE

In [ ]:
# delete
query = "DELETE from summarization;"
conn = ms_sql_con()  
#cursor = conn.cursor()
#cursor.execute(query)

In [39]:
tx = "Statesman's Dilemma (Стандард) (США) (английский) (на английском языке) (в переводе на русский) (с английской буквы «С» — «С») (СССР) (далее — США) (Атлант) (КНР) (Англия) (Британия) (Нью-Йорк) (Великобритания, Канада) (Боснийская) (Монголия)"

In [42]:
tx.replace("'","").replace('"','')

'Statesmans Dilemma (Стандард) (США) (английский) (на английском языке) (в переводе на русский) (с английской буквы «С» — «С») (СССР) (далее — США) (Атлант) (КНР) (Англия) (Британия) (Нью-Йорк) (Великобритания, Канада) (Боснийская) (Монголия)'

In [9]:
side = 0
query = "SELECT text, source_id from transcribations where "
query += " side="+str(side)+" and "
#query += " record_date = '"+str(record_date)+"' and "
query += " linkedid = '1622799198.236534';"
text_df = read_sql(query)
phrases_count = len(text_df)
text_full = ', '.join([row.text for _id, row in text_df.iterrows()])
text_full, phrases_count, min(text_df.source_id)

('здравствуйте скажите пожалуйста мастера можно вызвать по ремонту холодильников сна, москве, стянул, он бы уже давно да, двух, он там где морозильник не сама м-ра телка вот обычно помещаю стал работать как морозилку, да да да, о нет магнитики разброс, вверху справа но мне наверно старая ещё три семнадцать, восемь девятьсот девять, девятьсот восемьдесят один пятьдесят один, семьдесят, галина, улица онежская, дом тридцать пять, корпус два квартира двести пятьдесят семь, третий подъезд пятый этаж без лифта, водный стадион, да, сто пятьдесят девять сорок пять восемьдесят четыре, хорошо спасибо',
 21,
 1)

In [13]:
# update summarization field

In [36]:
query = "SELECT distinct"
query += " summarization.record_date, summarization.linkedid, transcribations.source_id"
query += " from summarization"
query += " inner join transcribations on"
query += " summarization.record_date = transcribations.record_date and"
query += " summarization.linkedid = transcribations.linkedid"
query += " where "
query += " isnull(summarization.source_id,-1)=-1;"
df = read_sql(query)
len(df)

0

In [ ]:
# 13518

In [35]:
prgBar = IntProgress(min = 0, max = len(df))
display(prgBar)
for _id, row in df.iterrows():
    query = "update summarization set source_id = '"+str(row.source_id)+"' where "
    query += " record_date = '"+str(row.record_date)+"' and"
    query += " linkedid = '"+str(row.linkedid)+"';"    
    conn = ms_sql_con()  
    cursor = conn.cursor()
    cursor.execute(query)   
    prgBar.value = prgBar.value + 1

IntProgress(value=0, max=13517)

In [19]:
query = "SELECT * from summarization where "
query += " load_msk = 1;"
df = read_sql(query)
len(df)

52